FASE 3: Extracción de Detalles de Actores con Selenium

In [1]:
# Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


Del csv con todas nuestras peliculas, nos quedamos solo con la columna de los ids (la comprobamos):

In [2]:
df_pelis = pd.read_csv("peliculas_combinadas.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'peliculas_combinadas.csv'

In [3]:
id_IMDB= df_pelis["ID"]
id_IMDB[:5]

0    tt10225380
1    tt10844184
2    tt10928904
3    tt11078340
4    tt11374514
Name: ID, dtype: object

In [4]:
df_pelis[2:3]


,ID,Nombre,Tipo,Año de Estreno,Mes de Estreno,Genero
2,tt10928904,The Legend of Johnny Jones,Movie,2024,10,Accion


Comenzamos con SELENIUM!Por ahora, solo con una pelicula:

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/title/tt0448115/")

sleep(3)

driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(2)

driver.maximize_window()

driver.execute_script("window.scrollTo(0,2000)")
sleep(2)

dicc_actors = {"nombre_act": [], "conocide_por": []}
#me voy al primer actor/actriz

driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.sc-bfec09a1-7.gWwKlt > a").click()
sleep(3)
dicc_actors= {"Nombre actr" : [], 
              "conocide_por": [],
              "año_nacimiento":[],
              "profesión": [],
              "premios": []}

nombre_act = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > h1 > span").text
dicc_actors["Nombre actr"].append(nombre_act)

sleep(5)

conocide_por= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
dicc_actors["conocide_por"].append(conocide_por)



sleep(5)

conocide_por2=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
dicc_actors["conocide_por"].append(conocide_por2)
print("He pasado por aqui")

sleep(7)

#ESTE NO ME FUNCIONA - año de nacimiento  
##año_nacimiento= driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(20) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(5) > div > ul > li > span").text

##limpiamos lo que nos devuelve despues de - para solo quedarnos con la fecha de nacimiento 
##partes= año_nacimiento.split(" - ")
##primera_parte=partes[0]
#print(primera_parte)

#año_nacimiento.append(primera_parte)

#que hace 
sleep(4)
driver.execute_script("window.scrollTo(0,4000)")

sleep(5)
profesion= driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.ipc-title.ipc-title--base.ipc-title--title.ipc-title--on-textPrimary.sc-56ff4734-4.kSHHyf.filmo-section-producer > hgroup").text
dicc_actors["profesión"].append(profesion)


#que_hace2=driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.ipc-title.ipc-title--base.ipc-title--title.ipc-title--on-textPrimary.sc-56ff4734-4.kSHHyf.filmo-section-director > hgroup").text
#que_hace.append(que_hace2)






sleep(5)

driver.close()



He pasado por aqui


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.ipc-title.ipc-title--base.ipc-title--title.ipc-title--on-textPrimary.sc-56ff4734-4.kSHHyf.filmo-section-producer > hgroup"}
  (Session info: chrome=124.0.6367.63); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7AEA61502+60802]
	(No symbol) [0x00007FF7AE9DAC02]
	(No symbol) [0x00007FF7AE897CE4]
	(No symbol) [0x00007FF7AE8E6D4D]
	(No symbol) [0x00007FF7AE8E6E1C]
	(No symbol) [0x00007FF7AE92CE37]
	(No symbol) [0x00007FF7AE90ABBF]
	(No symbol) [0x00007FF7AE92A224]
	(No symbol) [0x00007FF7AE90A923]
	(No sym

In [ ]:
#xpath "//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/ul/li[3]/div/ul/li[1]/a"

#css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(20) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(3) > div > ul > li.ipc-inline-list__item.test-class-react > a"
#"css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(20) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(5) > div > ul > li > span"


Creamos una lista vacía para los primeros 10 actores/actrices (link desde la página principal). donde solo cambia el numero, {i}

In [22]:
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/title/tt0448115/")

sleep(3)

driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(2)

driver.maximize_window()

driver.execute_script("window.scrollTo(0,2000)")
sleep(2)

dicc_actors = {"nombre_act": [], "conocide_por": []}

#me voy al primer actor/actriz
#driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.sc-bfec09a1-7.gWwKlt > a").click()
#sleep(3)

lista_url=[]
for i in range (1,11):
    lista_url.append(driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i})").text)

driver.close()


Extraemos los 10 primeros nombres de los actores y actrices de la película.

In [36]:
lista_url
nombres_10 = [nombre.split('\n')[0] for nombre in lista_url]
nombres_10


['Zachary Levi',
 'Mark Strong',
 'Asher Angel',
 'Jack Dylan Grazer',
 'Adam Brody',
 'Djimon Hounsou',
 'Faithe Herman',
 'Meagan Good',
 'Grace Caroline Currey',
 'Michelle Borth']

In [34]:
nombres_10.append(dicc_actors["nombre_act"])

#Creo que hay que crear otro bucle for para que appendee

In [ ]:
#COMPROBANDO css selectores- actores
#"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1)"
#"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2)"
#"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(3)"
#num10
##__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page2.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(10)

In [22]:
####parte añadida por lydia, nada claro,lo dejo de momento por si acaso pero creo que no va a servir..
# iteramos sobre la lista de elementos
for elemento in conocide_por:
    # Acceder al texto dentro de cada elemento y agregarlo al diccionario
    texto_elemento = elemento.text
    dicc_actors["conocide_por"].append(texto_elemento)

# Imprimir el diccionario con los actores
print(dicc_actors)



titulos = []

# Iteramos sobre la lista de elementos
for elemento in conocide_por:
    # Acceder al texto dentro de cada elemento y agregarlo a la lista de títulos
    titulo_elemento = elemento.text
    titulos.append(titulo_elemento)

# Imprimir la lista de títulos
print(titulos)








{'nombre_act': [], 'conocide_por': [[<selenium.webdriver.remote.webelement.WebElement (session="2233b7d8995ac68a79291a4b3f6aac02", element="f.56DF74BB773B3C8ADF456FFECAE83645.d.8DE69033094D88C6D477C1A3E87CFC28.e.186")>], '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de TV\nChuck Bartowski\n2007–2012 • 91 episodios\nEnredados\n7,7\nFlynn Rider(voz)\n2010\nThor: El mundo oscuro\n6,8\nFandral\n2013', '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de TV\nChuck Bartowski\n2007–2012 • 91 episodios\nEnredados\n7,7\nFlynn Rider(voz)\n2010\nThor: El mundo oscuro\n6,8\nFandral\n2013', '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de TV\nChuck Bartowski\n2007–2012 • 91 episodios\nEnredados\n7,7\nFlynn Rider(voz)\n2010\nThor: El mundo oscuro\n6,8\nFandral\n2013', '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de TV\nChuck Bartowski\n2007–2012 • 91 episodios\nEnredados\n7,7\nFlynn Rider(voz)\n2010\nThor: El mundo oscuro\n6,8\nFandral\n2013', '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de 

In [ ]:
for pelicula in web_peli:
    try:
        url_imdb= f"url{id}"
    
    except:
        pass

In [ ]:
#para buscar premios
driver.find_element("link text", "Premios").click()